In [1]:
import orbax.checkpoint


In [1]:
from clu import metrics
from flax.training import train_state
from flax import struct
import random
from flax import struct
from clu import metrics
from flax.training import train_state
import optax

import jax
import jax.numpy as jnp
from jax import grad, random
import optax
import orbax.checkpoint as ocp
import wandb

from models import unigram
from data import load_data
from const import VOCAB_SIZE
from flax.training import orbax_utils

EMBEDDING_DIM = 64
BATCH_SIZE = 64
CKPT_DIR = '/home/trung/transformer/ckpt/unigram'

@struct.dataclass
class Metrics(metrics.Collection):
    accuracy: metrics.Accuracy
    loss: metrics.Average.from_output('loss')

class TrainState(train_state.TrainState):
    metrics: Metrics

def create_train_state(module, rng, learning_rate, momentum):
    """Creates an initial `TrainState`."""
    dummy = random.randint(key1, minval=0, maxval=10, shape=(10,))
    params = module.init(rng, dummy)['params'] # initialize parameters by passing a template image
    tx = optax.sgd(learning_rate, momentum)
    return TrainState.create(
      apply_fn=module.apply, params=params, tx=tx,
      metrics=Metrics.empty())

@jax.jit
def train_step(state, batch):
    """Train for a single step."""
    def loss_fn(params):
        logits = state.apply_fn({'params': params}, batch['x'])
        loss = optax.softmax_cross_entropy_with_integer_labels(
            logits=logits, labels=batch['y']).mean()
        return loss
    grad_fn = jax.grad(loss_fn)
    grads = grad_fn(state.params)
    state = state.apply_gradients(grads=grads)
    return state 

@jax.jit 
def compute_metrics(*, state, batch):
    logits = state.apply_fn({'params': state.params}, batch['x'])
    loss = optax.softmax_cross_entropy_with_integer_labels(
        logits=logits, labels=batch['y']).mean()
    metric_updates = state.metrics.single_from_model_output(
    logits=logits, labels=batch['y'], loss=loss)
    metrics = state.metrics.merge(metric_updates)
    state = state.replace(metrics=metrics)
    return state

@jax.jit
def pred_step(state, x, key, T=10):
    logits = state.apply_fn({'params': state.params}, x)
    return jax.random.categorical(key, logits/T, axis=1)

        
model = unigram.UnigramModel(vocab_size=VOCAB_SIZE, embedding_dim=64)
key1, key2 = random.split(random.key(0))


2024-03-03 22:56:49.854918: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
learning_rate = 0.01
momentum = 0.9

init_rng = jax.random.key(0)
state = create_train_state(model, init_rng, learning_rate, momentum)


In [3]:
from clu import metrics
from flax.training import train_state
from flax import struct
import random
from flax import struct
from clu import metrics
from flax.training import train_state
import optax

import jax
import jax.numpy as jnp
from jax import grad, random
import optax
import orbax.checkpoint
import wandb

from models import unigram
from data import load_data
from const import VOCAB_SIZE
from flax.training import orbax_utils


options = ocp.CheckpointManagerOptions(max_to_keep=3)
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
checkpoint_manager = orbax.checkpoint.CheckpointManager(
    CKPT_DIR, options=options)



In [8]:
ckpt_state = {
    'model': state,
    'config': {'step': 0}
}
abstract_state = jax.tree_util.tree_map(ocp.utils.to_shape_dtype_struct, ckpt_state)
abstract_state

{'config': {'step': 0},
 'model': TrainState(step=0, apply_fn=<bound method Module.apply of UnigramModel(
     # attributes
     vocab_size = 40478
     embedding_dim = 64
 )>, params={'Dense_0': {'bias': ShapeDtypeStruct(shape=(40478,), dtype=float32, sharding=SingleDeviceSharding(device=cuda(id=0))), 'kernel': ShapeDtypeStruct(shape=(64, 40478), dtype=float32, sharding=SingleDeviceSharding(device=cuda(id=0)))}, 'unigram': {'embedding': ShapeDtypeStruct(shape=(40478, 64), dtype=float32, sharding=SingleDeviceSharding(device=cuda(id=0)))}}, tx=GradientTransformationExtraArgs(init=<function chain.<locals>.init_fn at 0x7f4b5fea9bd0>, update=<function chain.<locals>.update_fn at 0x7f4b5fea9e10>), opt_state=(TraceState(trace={'Dense_0': {'bias': ShapeDtypeStruct(shape=(40478,), dtype=float32, sharding=SingleDeviceSharding(device=cuda(id=0))), 'kernel': ShapeDtypeStruct(shape=(64, 40478), dtype=float32, sharding=SingleDeviceSharding(device=cuda(id=0)))}, 'unigram': {'embedding': ShapeDtypeSt

In [10]:
restored_state = checkpoint_manager.restore(checkpoint_manager.latest_step(), args=ocp.args.StandardRestore(abstract_state))

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai-gpt")



Array([[19874]], dtype=int32)

In [68]:
def get_next_word(s):
    encoded = tokenizer.encode(s, return_tensors="jax")[0]
    pred = pred_step(restored_state['model'], encoded, init_rng, T=1)
    return tokenizer.decode(pred)

In [81]:
get_next_word("weather")

'conditions'

In [14]:
@struct.dataclass
class Metrics(metrics.Collection):
    accuracy: metrics.Accuracy
    loss: metrics.Average.from_output('loss')

class TrainState(train_state.TrainState):
    metrics: Metrics

def create_train_state(module, rng, learning_rate, momentum):
    """Creates an initial `TrainState`."""
    dummy = random.randint(key1, minval=0, maxval=10, shape=(10,))
    params = module.init(rng, dummy)['params'] # initialize parameters by passing a template image
    tx = optax.sgd(learning_rate, momentum)
    return TrainState.create(
      apply_fn=module.apply, params=params, tx=tx,
      metrics=Metrics.empty())


ValueError: Unsupported value_metadata class: <class 'orbax.checkpoint.value_metadata.Metadata'>.